In [1]:
!nvidia-smi

Mon Oct 24 01:09:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Graphics...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   62C    P3     6W /  N/A |   1911MiB /  3913MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
import os
os.getcwd()

'/home/yyj/sahi/demo'

In [19]:
# arrange an instance segmentation model for test
from sahi.utils.yolov5 import (
    download_yolov5s6_model,
)

# import required functions, classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.utils.file import download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict
from IPython.display import Image
from pathlib import Path
import json

In [20]:
source_image_dir = "../resources/FLL_VAL/images/"
image_files = sorted([fn for fn in os.listdir(source_image_dir) if fn.endswith("jpg")])

## Standard Prediction

In [21]:
DSL_STANDARD_JSON_FILE = '../resources/DSL/result_FL221022_0.25_standard_pred.json'

with open(DSL_STANDARD_JSON_FILE, 'r') as f:
    datas = json.load(f)

In [22]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

@interact(index=(0, len(image_files)-1))
def show_sample(index=0):
    infos = [ data for data in datas if data['image_id'] == index ]
    image_path = os.path.join(source_image_dir, image_files[index])
    image = cv2.imread(image_path)    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for info in infos:
        bbox = list(map(int, info['bbox']))
        category_name =  info['category_name']
        conf = info['score']
        label = f"{category_name} {conf:.2f}"
        
        cv2.rectangle(image,
                      bbox[:2],
                      [ bbox1 + bbox2 for bbox1, bbox2 in zip(bbox[:2], bbox[2:])],
                      color=(255, 0, 0),
                      thickness=2)
        cv2.putText(
            image,
            label,  
            [bbox[0], bbox[1]-10],
            0,
            1,
            (255, 255, 255),
            thickness=3,
        )
        
    plt.figure(figsize=(16,16))
    plt.imshow(image)
    plt.axis('off')
    plt.show()


interactive(children=(IntSlider(value=0, description='index', max=4896), Output()), _dom_classes=('widget-inte…

## Slice Prediction

In [16]:
DSL_SLICE_JSON_FILE = '../resources/DSL/result_slice_pred.json'

with open(DSL_SLICE_JSON_FILE, 'r') as f:
    datas = json.load(f)

In [17]:
custom_slice_x_start=640
custom_slice_y_start=360
slice_size=512

In [18]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

@interact(index=(0, len(image_files)-1))
def show_sample(index=0):
    infos = [ data for data in datas if data['image_id'] == index ]
    image_path = os.path.join(source_image_dir, image_files[index])
    image = cv2.imread(image_path)    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    for info in infos:
        bbox = list(map(int, info['bbox']))
        category_name =  info['category_name']
        conf = info['score']
        label = f"{category_name} {conf:.2f}"
        
        cv2.rectangle(image,
                      bbox[:2],
                      [ bbox1 + bbox2 for bbox1, bbox2 in zip(bbox[:2], bbox[2:])],
                      color=(255, 0, 0),
                      thickness=2)
        
        cv2.rectangle(image,
                      [custom_slice_x_start, custom_slice_y_start],
                      [custom_slice_x_start+slice_size, custom_slice_y_start+slice_size],
                      color=(0, 255, 0),
                      thickness=2)
        
        cv2.putText(
            image,
            label,  
            [bbox[0], bbox[1]-10],
            0,
            1,
            (255, 255, 255),
            thickness=3,
        )
        
    plt.figure(figsize=(16,16))
    plt.imshow(image)
    plt.axis('off')
    plt.show()


interactive(children=(IntSlider(value=0, description='index', max=4896), Output()), _dom_classes=('widget-inte…

## Compare Standard and Slice

In [23]:
DSL_STANDARD_JSON_FILE = '../resources/DSL/result_FL221022_0.25_standard_pred.json'
DSL_SLICE_JSON_FILE = '../resources/DSL/result_slice_pred.json'

with open(DSL_STANDARD_JSON_FILE, 'r') as f:
    standard_datas = json.load(f)

with open(DSL_SLICE_JSON_FILE, 'r') as f:
    slice_datas = json.load(f)

In [24]:
from ipywidgets import interact
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

    
@interact(index=(0, len(image_files)-1))
def show_sample(index=0):
    standard_infos = [ data for data in standard_datas if data['image_id'] == index ]
    slice_infos = [ data for data in slice_datas if data['image_id'] == index ]
    
    image_path = os.path.join(source_image_dir, image_files[index])
    image1 = cv2.imread(image_path)    
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    
    for info in standard_infos:
        bbox = list(map(int, info['bbox']))
        category_name =  info['category_name']
        conf = info['score']
        label = f"{category_name} {conf:.2f}"
        
        cv2.rectangle(image1,
                      bbox[:2],
                      [ bbox1 + bbox2 for bbox1, bbox2 in zip(bbox[:2], bbox[2:])],
                      color=(255, 0, 0),
                      thickness=2)
        
        cv2.putText(
            image1,
            label,  
            [bbox[0], bbox[1]-10],
            0,
            1,
            (255, 255, 255),
            thickness=3,
        )
    
    image_path = os.path.join(source_image_dir, image_files[index])
    image2 = cv2.imread(image_path)    
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    
    
    for info in slice_infos:
        bbox = list(map(int, info['bbox']))
        category_name =  info['category_name']
        conf = info['score']
        label = f"{category_name} {conf:.2f}"
        
        cv2.rectangle(image2,
                      bbox[:2],
                      [ bbox1 + bbox2 for bbox1, bbox2 in zip(bbox[:2], bbox[2:])],
                      color=(255, 0, 0),
                      thickness=2)
        
        cv2.rectangle(image2,
                      [custom_slice_x_start, custom_slice_y_start],
                      [custom_slice_x_start+slice_size, custom_slice_y_start+slice_size],
                      color=(0, 255, 0),
                      thickness=2)
        
        cv2.putText(
            image2,
            label,  
            [bbox[0], bbox[1]-10],
            0,
            1,
            (255, 255, 255),
            thickness=3,
        )
        
    fig, axes = plt.subplots(1, 2, figsize=(32,32)) 
    plt.subplots_adjust(wspace=0.05, hspace=0.05)
    
    axes[0].axis('off')
    axes[1].axis('off')
    
    axes[0].imshow(image1)
    axes[1].imshow(image2)
    
    plt.axis('off')
    plt.show()


interactive(children=(IntSlider(value=0, description='index', max=4896), Output()), _dom_classes=('widget-inte…

## Slice Prediction (SAHI)

In [ ]:
custom_slice_x_start=640
custom_slice_y_start=360
custom_slice_mode=2
slice_size=512

In [ ]:
fll_model_221014_path = '../resources/models/221014/best.pt'

fll_model_221014 = AutoDetectionModel.from_pretrained(
    model_type='yolov5',
    model_path=fll_model_221014_path,
    confidence_threshold=0.25,
    device="cuda:0"
)

model = fll_model_221014
model_path = fll_model_221014_path

In [ ]:
from sahi.utils.cv import Colors
import numpy as np
import copy

def visualize_object_predictions(
    image: np.array,
    object_prediction_list,
    rect_th: int = None,
    text_size: float = None,
    text_th: float = None,
    color: tuple = None,
):
    """
    Visualizes prediction category names, bounding boxes over the source image
    and exports it to output folder.
    Arguments:
        object_prediction_list: a list of prediction.ObjectPrediction
        rect_th: rectangle thickness
        text_size: size of the category name over box
        text_th: text thickness
        color: annotation color in the form: (0, 255, 0)
        output_dir: directory for resulting visualization to be exported
        file_name: exported file will be saved as: output_dir+file_name+".png"
        export_format: can be specified as 'jpg' or 'png'
    """
    # deepcopy image so that original is not altered
    image = copy.deepcopy(image)
    # select predefined classwise color palette if not specified
    if color is None:
        colors = Colors()
    else:
        colors = None
    # set rect_th for boxes
    rect_th = rect_th or max(round(sum(image.shape) / 2 * 0.001), 1)
    # set text_th for category names
    text_th = text_th or max(rect_th - 1, 1)
    # set text_size for category names
    text_size = text_size or rect_th / 3
    # add bbox and mask to image if present
    for object_prediction in object_prediction_list:
        # deepcopy object_prediction_list so that original is not altered
        object_prediction = object_prediction.deepcopy()

        bbox = object_prediction.bbox.to_voc_bbox()
        category_name = object_prediction.category.name
        score = object_prediction.score.value

        # set color
        if colors is not None:
            color = colors(object_prediction.category.id)
        # visualize masks if present
        if object_prediction.mask is not None:
            # deepcopy mask so that original is not altered
            mask = object_prediction.mask.bool_mask
            # draw mask
            rgb_mask = apply_color_mask(mask, color)
            image = cv2.addWeighted(image, 1, rgb_mask, 0.4, 0)
        # set bbox points
        p1, p2 = (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3]))
        # visualize boxes
        cv2.rectangle(
            image,
            p1,
            p2,
            color=color,
            thickness=rect_th
        )
        # arange bounding box text location
        label = f"{category_name} {score:.2f}"
        w, h = cv2.getTextSize(label, 0, fontScale=text_size, thickness=text_th)[0]  # label width, height
        outside = p1[1] - h - 3 >= 0  # label fits outside box
        p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
        # add bounding box text
        cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(
            image,
            label,
            (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
            0,
            text_size,
            (255, 255, 255),
            thickness=text_th,
        )
        
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
    return image

In [ ]:
from ipywidgets import interact
from sahi.slicing import slice_image
import matplotlib.pyplot as plt
%matplotlib inline

@interact(index=(0, len(image_files)-1), slice_size=(0, 512), overlap_ratio=(0, 0.5, 0.05), only_full_inference=(0,1))
def show_sample(index=0, slice_size=512, overlap_ratio=0.2, only_full_inference=0):
    image_file = image_files[index]
    image_path = os.path.join(source_image_dir, image_file)
    image = cv2.imread(image_path)
    
    if not only_full_inference:
        slice_result = slice_image(image_path, 
                                  slice_width=slice_size,
                                  slice_height=slice_size,
                                  overlap_height_ratio=overlap_ratio,
                                  overlap_width_ratio=overlap_ratio,
                                  custom_slice_mode=custom_slice_mode,
                                  custom_slice_x_start=custom_slice_x_start,
                                  custom_slice_y_start=custom_slice_y_start,
                                  verbose=1)

        for start_pixel in slice_result.starting_pixels:
            cv2.rectangle(image,
                          start_pixel,
                          [s1+s2 for s1, s2 in zip(start_pixel,[slice_size,slice_size])],
                          color=(255, 255, 0),
                          thickness=2)
        
        result = get_sliced_prediction(image_path,
                                       model,
                                       slice_height=slice_size,
                                       slice_width=slice_size,
                                       postprocess_match_threshold=0.5,
                                       overlap_height_ratio=overlap_ratio,
                                       overlap_width_ratio=overlap_ratio,
                                       custom_slice_mode=custom_slice_mode,
                                       postprocess_type="GREEDYNMM",
                                       custom_slice_x_start=custom_slice_x_start,
                                       custom_slice_y_start=custom_slice_y_start
                                      )
    else:
        result = get_prediction(image_path, model)
    
    canvas = visualize_object_predictions(image, result.object_prediction_list)
    plt.figure(figsize=(16,16))
    plt.imshow(canvas)
    plt.axis('off')
    plt.show()